In [2]:
%pip install Sastrawi


  Using cached Sastrawi-1.0.1-py2.py3-none-any.whl.metadata (909 bytes)
Using cached Sastrawi-1.0.1-py2.py3-none-any.whl (209 kB)


In [3]:
import pandas as pd
import re
import PyPDF2
import nltk
import os
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import spacy
import spacy_stanza
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import csv

In [4]:
nltk.download('punkt')
nlp = spacy_stanza.load_pipeline("id") 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khalf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-06-02 14:39:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-06-02 14:39:43 INFO: Loading these models for language: id (Indonesian):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | icon_charlm  |
| depparse     | gsd_charlm   |

2024-06-02 14:39:43 WARNING: GPU requested, but is not available!
2024-06-02 14:39:43 INFO: Using device: cpu
2024-06-02 14:39:43 INFO: Loading: tokenize
2024-06-02 14:39:47 INFO: Loading: mwt
2024-06-02 14:39:47 INFO: Loading: pos
2024-06-02 14:39:49 INFO: Loading: lemma
2024-06-02 14:39:49 INFO: Loading: constituency
2024-06-02 14:39:52 INFO: Loading: depparse
2024-06-02 14:39:53 INFO: Done loading processors!


In [9]:
def extract_text_from_pdf(pdf_path, exclude_patterns=None, start_page=5, end_page=11):
    text = ""
    if exclude_patterns is None:
        exclude_patterns = []

    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        
        # Ensure the start and end page numbers are within the valid range
        start_page = max(0, start_page - 1)  # Pages are 0-indexed in PyPDF2
        end_page = min(num_pages, end_page)  # Ensure end_page does not exceed total pages
        
        for page_num in range(start_page, end_page):
            page = reader.pages[page_num]
            page_text = page.extract_text()

            # Exclude specified patterns from the extracted text
            for pattern in exclude_patterns:
                page_text = page_text.replace(pattern, '')

            text += page_text
    
    return text


In [10]:
# Define File Path
pdf_path = "C:/Users/khalf/Code/NLP/draft_ruu_kuhp_final.pdf"

# Extract File
extract_text = extract_text_from_pdf(pdf_path)
print(extract_text)

5 
 BAB II  
TINDAK PIDANA DAN PERTANGGUNGJAWABAN PIDANA  
 
Bagian Kesatu  
Tindak Pidana  
 
Paragraf 1  
Umum  
 
Pasal 12  
(1) Tindak Pidana merupakan  perbuatan yang oleh peraturan perundang -
undangan diancam dengan sanksi pidana dan/atau tindakan . 
(2)  Untuk dinyatakan sebagai Tindak Pidana , suatu perbuatan yang diancam 
sanksi pidana dan/atau tindakan oleh peraturan perundang -undangan 
harus bersifat melawan hukum atau bertentangan dengan hukum yang 
hidup dalam masyarakat.  
(3) Setiap Tindak Pidana  selalu bersifat melawan hukum, kecuali ada alasan 
pembenar.  
 
Paragraf 2  
Permufakatan Jahat  
 
Pasal 13  
(1)  Permufakatan jahat  terjadi jika 2 (dua) or ang atau lebih bersepakat untuk 
melakukan Tindak Pidana . 
(2) Permufakata n jahat melakukan Tindak Pidana  dipidana jika ditentukan 
secara tegas dalam Undang -Undang.  
(3) Pidana untuk permufakata n jahat melakukan Tindak Pidana  paling banyak 
1/3 (satu per tiga) dari maksimum ancaman p idana pokok untuk Tindak 


In [20]:
def classify_text(text):
    bab_pattern = r'BAB\s+[IVXLCDM]+\s*.*'
    bagian_pattern = r'Bagian\s+[A-Za-z]+\s*.*'
    paragraf_pattern = r'Paragraf\s+\d+\s*.*'
    pasal_pattern = r'Pasal\s+\d+\s*.*'
    ayat_pattern = r'\(\d+\)\s*.*'

    data = []
    current_bab = current_bagian = current_paragraf = current_pasal = None

    lines = text.split('\n')
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        if re.match(bab_pattern, line):
            current_bab = line
            current_bagian = current_paragraf = current_pasal = None
        
        elif re.match(bagian_pattern, line):
            current_bagian = line
            current_paragraf = current_pasal = None
        
        elif re.match(paragraf_pattern, line):
            current_paragraf = line
            current_pasal = None
        
        elif re.match(pasal_pattern, line):
            current_pasal = line
            i += 1
            next_line = lines[i].strip() if i < len(lines) else ""
            
            # Check if the next line is an Ayat
            if re.match(ayat_pattern, next_line):
                while i < len(lines) and re.match(ayat_pattern, lines[i].strip()):
                    ayat_line = lines[i].strip()
                    data.append({
                        'BAB': current_bab,
                        'Bagian': current_bagian,
                        'Paragraf': current_paragraf,
                        'Pasal': current_pasal,
                        'Ayat': ayat_line
                    })
                    i += 1
            else:
                # No Ayat, just text under Pasal
                pasal_text = current_pasal + " " + next_line
                while i < len(lines) and not re.match(pasal_pattern, lines[i].strip()):
                    i += 1
                    if i < len(lines):
                        pasal_text += " " + lines[i].strip()
                data.append({
                    'BAB': current_bab,
                    'Bagian': current_bagian,
                    'Paragraf': current_paragraf,
                    'Pasal': current_pasal,
                    'Ayat': pasal_text
                })
                continue

        i += 1

    return pd.DataFrame(data)

In [21]:
Classifed_Text = classify_text(extract_text)
Classifed_Text.to_csv('classified to pasal.csv', index = False)
print(Classifed_Text.head)


<bound method NDFrame.head of        BAB         Bagian    Paragraf      Pasal  \
0   BAB II  Bagian Kesatu  Paragraf 1   Pasal 12   
1   BAB II  Bagian Kesatu  Paragraf 2   Pasal 13   
2   BAB II  Bagian Kesatu  Paragraf 2   Pasal 14   
3   BAB II  Bagian Kesatu  Paragraf 2   Pasal 15   
4   BAB II  Bagian Kesatu  Paragraf 2   Pasal 16   
5   BAB II  Bagian Kesatu  Paragraf 2   Pasal 17   
6   BAB II  Bagian Kesatu  Paragraf 2   Pasal 18   
7   BAB II  Bagian Kesatu  Paragraf 2   Pasal 19   
8   BAB II  Bagian Kesatu  Paragraf 2   Pasal 20   
9   BAB II  Bagian Kesatu  Paragraf 2   Pasal 21   
10  BAB II  Bagian Kesatu  Paragraf 2   Pasal 22   
11  BAB II  Bagian Kesatu  Paragraf 2   Pasal 23   
12  BAB II  Bagian Kesatu  Paragraf 7   Pasal 24   
13  BAB II  Bagian Kesatu  Paragraf 7   Pasal 25   
14  BAB II  Bagian Kesatu  Paragraf 7   Pasal 26   
15  BAB II  Bagian Kesatu  Paragraf 7   Pasal 27   
16  BAB II  Bagian Kesatu  Paragraf 7   Pasal 28   
17  BAB II  Bagian Kesatu  Paragra